In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('..','..','util'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [19]:
import types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [23]:
import os
import pandas as pd

from MLUtil import fetch_titanic_data

# loading data from MLutil.fetch_titanic_data
train_data = fetch_titanic_data("train.csv")
test_data = fetch_titanic_data("test.csv")

train_data.head()

ModuleNotFoundError: No module named 'util'

In [1]:
train_data.info()

NameError: name 'train_data' is not defined

Null values in age feature. Cabin feature has 

In [ ]:
train_data.describe()

In [ ]:
corr_matrix = train_data.corr()
corr_matrix["Survived"].sort_values(ascending=False)

plotting correlation matrix

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["Survived", "Cabin", "Embarked","Fare", "Age", "Pclass", "Sex"]
scatter_matrix(train_data[attributes], figsize = (14, 14))

In [ ]:
train_data["Survived"].value_counts(),train_data["Pclass"].value_counts(), train_data["Sex"].value_counts(), train_data["Embarked"].value_counts()